In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')
import calitp_data_analysis.magics
import update_vars_index
import pandas as pd
import geopandas as gpd
from siuba import *
from shared_utils import catalog_utils, rt_utils
import speedmap_utils

analysis_date = update_vars_index.ANALYSIS_DATE
catalog = catalog_utils.get_catalog('gtfs_analytics_data')

In [ ]:
## parameters cell
organization_source_record_id = 'rec5ome04BbA9uf4y'

In [ ]:
%%capture
speedmap_segs, this_shn = speedmap_utils.read_segments_shn(organization_source_record_id=organization_source_record_id)

In [ ]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

In [ ]:
%%capture_parameters
organization_name = speedmap_segs.organization_name.iloc[0]
organization_name

In [ ]:
import importlib
importlib.reload(speedmap_utils)
importlib.reload(update_vars_index)

# {organization_name}

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for {organization_name}, with a map each for the morning peak, midday, and afternoon peak periods on {human_date}.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel. Thicker segments indicate more frequent transit routes.
* State Highway Network routes are outlined in gray.
* Hover over, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

## AM Peak

In [ ]:
%%capture
link = speedmap_utils.map_time_period(district_gdf=this_shn, speedmap_segs=speedmap_segs, analysis_date=analysis_date,
                              time_of_day='AM Peak', map_type='new_speedmap')

In [ ]:
speedmap_utils.render_spa_link(link)

In [ ]:
speedmap_utils.display_spa_map(link)

In [ ]:
%%capture
link = speedmap_utils.map_time_period(district_gdf=this_shn, speedmap_segs=speedmap_segs, analysis_date=analysis_date,
                              time_of_day='AM Peak', map_type='new_speed_variation')

In [ ]:
speedmap_utils.render_spa_link(link)

In [ ]:
speedmap_utils.display_spa_map(link)

In [ ]:
speedmap_utils.chart_speeds_by_time_period(speedmap_segs)